In [59]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence


In [60]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [61]:
torch.set_default_tensor_type(torch.FloatTensor)

In [62]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [63]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [64]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NSW"]
data_file = 'gru_inputs_simple_v6_updated.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True, date_filter=date)

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]
(1273739, 24)
20
Latest date = 2023-06-14 00:00:00
size after state filter (316570, 26)
(316570, 26)
Latest date = 2023-06-14 00:00:00
num_features_per_dog=20


  0%|          | 0/7180 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/11947 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
 41%|████▏     | 4941/11947 [00:14<00:18, 376.68it/s]

index 7 is out of bounds for dimension 0 with size 5


 63%|██████▎   | 7523/11947 [00:23<00:23, 186.67it/s]

index 7 is out of bounds for dimension 0 with size 7


 79%|███████▉  | 9411/11947 [00:27<00:05, 479.65it/s]

index 5 is out of bounds for dimension 0 with size 3


100%|██████████| 11947/11947 [00:34<00:00, 347.24it/s]


number of races = 11947, number of unique dogs = 7181


In [65]:
races = raceDB.get_test_input([1,2,3])
races[0].race_dist

tensor([300.], device='cuda:0', dtype=torch.float64)

In [66]:
races[0].race_dist

tensor([300.], device='cuda:0', dtype=torch.float64)

In [67]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 0, Test examples 11947


In [68]:
raceDB.create_dogs_test_split_date()

100%|██████████| 7180/7180 [00:00<00:00, 166827.89it/s]


In [69]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 7181/7181 [00:01<00:00, 5767.26it/s]


In [70]:
all_dog_input = [list(d.races.values()) for d in raceDB.dogsDict.values() if d.races.values()]

In [71]:
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in all_dog_input]], enforce_sorted=False) 

In [72]:
packed_y = packed_y.to('cuda:0')

In [73]:
# model_name = 'curious-durian-408'
# model,hidden_state_dict = setup_model(model_name)

In [74]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [75]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [0, 0, 0]


In [76]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    # print(bi)
    # print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in tqdm(raceDB.dog_ids):
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in batch if x in dog.races.keys()]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7180 [00:00<?, ?it/s]

  0%|          | 0/7180 [00:00<?, ?it/s]

  0%|          | 0/7180 [00:00<?, ?it/s]

In [77]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [78]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [0, 0, 0]
Train examples [0, 0, 0]
Train examples [0, 0, 0]
Train examples [0, 0, 0]


In [79]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = ""#[pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_x_basic = ""#[pack_sequence([torch.stack(n,0) for n in [[z.stats for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = ""#pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
packed_y_basic = pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [80]:
raceDB.batches = {'num_batches':len(train_dogs),
                  'dogs':train_dogs,
                  'train_dog_input':train_dog_input,
                  'batch_races':batch_races,
                  'batch_races_ids':batch_races_ids,
                  'packed_x':packed_x,
                  'packed_x_basic':packed_x_basic,
                  'packed_y_basic':packed_y_basic,
                    'packed_y':packed_y}

In [81]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU - updates", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        input_size = raceDB.batches['packed_y'][0].data[0].shape[0]

      print(f"{input_size=}")

      # raceDB.batches['packed_y'][0] = raceDB.batches['packed_y'][0]._replace(data=)

      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_LN(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      criterion = nn.CrossEntropyLoss(reduction='none')
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      model = model.to(device)

      print(model)
      print(model.forward)

      # training_testing_gru.validate_model_v3(model,raceDB, criterion,epoch=0)
      # training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config, update=True)
      model.eval()
      training_testing_gru.validate_model_v3(model,raceDB, criterion,epoch=0)

      if sweep:
        raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [82]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                       'datafile':data_file,
                       'stats':raceDB.stats_cols,
                       'input_type':'basic',
                       'races':states,
                       'latest_date':raceDB.latest_date,
                       'num_layers':2,
                       'batch_size': 750,
                       'dropout': 0.3,
                       'epochs': 10_000,
                       'learning_rate': 0.0003,
                       'optimizer': 'adamW',
                       'f0_layer_size':128,
                       'f1_layer_size':64}

In [83]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [84]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [85]:
torch.cuda.empty_cache()

In [86]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='kind-wood-610', prev_model_version='1300')
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='youthful-butterfly-730', prev_model_version='700')

{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v6_updated.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-06-14T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0003, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64}
10000
{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v6_updated.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-06-14T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.

100%|██████████| 7181/7181 [00:00<00:00, 478296.66it/s]


filled =3845
empty  =3336
0.5354407464141484null_dog=0
GRUNetv3_LN(
  (gru): GRU(20, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)
<bound method GRUNetv3_LN.forward of GRUNetv3_LN(
  (gru): GRU(20, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_featu

ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
epoch,▁▁
flat_simple,▁▁
loss_val,▁▁
multibet outlay < 30,▁▁
multibet profit,▁▁
multibet profit < 30,▁▁
multibet profit < 30 sd,▁▁


In [29]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [30]:
race = raceDB.batches['batch_races'][0][2]
[d for d in race.dogs]

IndexError: list index out of range